Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [157]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Przygotowanie macierzy interakcji

In [158]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
productsDataPath = '../notebooks/data/v2/products.jsonl'
usersDataPath = '../notebooks/data/v2/users.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)
productsDF = pd.read_json(productsDataPath, lines=True)
usersDF = pd.read_json(usersDataPath, lines=True)

df = sessionsDF.drop(
    columns=["timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1

#TODO Below split does not make any sense. Proper split need to be aplied for final build, but this one works as fast split for testing.
#BEGIN OF SHIT
train, test = train_test_split(df, test_size=0.2, stratify=df['user_id'])

#idea is to make sure that all data in test set has at least two entries from given session. Data that not satisfies that criteria are appended to train set.
s = test['session_id'].value_counts()
res = test[test['session_id'].map(s) >= 2]
ret = test[test['session_id'].map(s) == 1]
test.sort_values(by='session_id', inplace=True)
res.sort_values(by='session_id', inplace=True)
# display(test)
# display(res)
#END OF SHIT

train.append(ret).sort_values(by='session_id', inplace=True)
interactionMatrixDF = pd.pivot_table(train,
                                     index="user_id",
                                     columns="product_id",
                                     values="count",
                                     aggfunc=np.sum,
                                     fill_value=0)
display(interactionMatrixDF)


C:\Users\Bartosz\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103,0,2,2,13,1,2,2,4,3,9,...,0,1,2,1,1,2,5,1,0,6
104,0,1,5,4,5,7,0,4,2,5,...,1,1,1,0,1,1,6,2,6,1
105,0,2,6,8,1,5,0,4,3,3,...,0,1,1,0,2,1,5,5,0,1
106,3,0,1,9,1,1,2,2,1,5,...,0,0,2,2,1,1,5,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,1,1,5,11,7,2,0,1,6,6,...,0,2,0,2,0,1,8,1,1,3
298,0,0,6,1,2,2,0,2,1,2,...,1,0,0,0,0,2,9,3,0,0
299,0,0,3,0,0,0,0,1,0,1,...,0,0,1,0,0,1,3,1,0,0


Normalizacja wartości w macierzy interakcji.

In [159]:
#TODO do not know if scaler is required at all
interactionMatrixDF = pd.DataFrame(
    preprocessing.MinMaxScaler().fit_transform(interactionMatrixDF))
display(interactionMatrixDF)

,0,1,2,3,4,5,6,7,8,9,...,309,310,311,312,313,314,315,316,317,318
0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000
1,0.000000,0.4,0.142857,0.481481,0.076923,0.181818,0.4,0.363636,0.3,0.5625,...,0.000000,0.166667,0.50,0.25,0.142857,0.50,0.192308,0.2,0.000000,0.857143
2,0.000000,0.2,0.357143,0.148148,0.384615,0.636364,0.0,0.363636,0.2,0.3125,...,0.333333,0.166667,0.25,0.00,0.142857,0.25,0.230769,0.4,0.857143,0.142857
3,0.000000,0.4,0.428571,0.296296,0.076923,0.454545,0.0,0.363636,0.3,0.1875,...,0.000000,0.166667,0.25,0.00,0.285714,0.25,0.192308,1.0,0.000000,0.142857
4,0.428571,0.0,0.071429,0.333333,0.076923,0.090909,0.4,0.181818,0.1,0.3125,...,0.000000,0.000000,0.50,0.50,0.142857,0.25,0.192308,0.6,0.000000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.142857,0.2,0.357143,0.407407,0.538462,0.181818,0.0,0.090909,0.6,0.3750,...,0.000000,0.333333,0.00,0.50,0.000000,0.25,0.307692,0.2,0.142857,0.428571
196,0.000000,0.0,0.428571,0.037037,0.153846,0.181818,0.0,0.181818,0.1,0.1250,...,0.333333,0.000000,0.00,0.00,0.000000,0.50,0.346154,0.6,0.000000,0.000000
197,0.000000,0.0,0.214286,0.000000,0.000000,0.000000,0.0,0.090909,0.0,0.0625,...,0.000000,0.000000,0.25,0.00,0.000000,0.25,0.115385,0.2,0.000000,0.000000
198,0.428571,0.8,0.428571,0.296296,0.076923,0.454545,0.0,0.363636,0.3,0.5000,...,0.333333,0.166667,0.00,0.00,0.571429,0.75,0.346154,0.2,0.000000,0.571429


Utworzenie słownika dla szybszego wyszukiwania.

In [160]:
idxNameDict = pd.Series(productsDF["product_id"].values,
                        index=productsDF.index).to_dict()
idxUserDict = pd.Series(usersDF["user_id"].values,
                        index=usersDF.index).to_dict()


Dekompozycja utworzonej macierzy na podmacierze ze względu na użytkowników i produkty.

In [161]:
from sklearn.decomposition import TruncatedSVD

#initial hiperparameters
epsilon = 1e-9
latentFactors = 10

#generate user latent features
userSVD = TruncatedSVD(n_components=latentFactors)
userFeatures = userSVD.fit_transform(interactionMatrixDF) + epsilon

#generate item latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF.transpose(
)) + epsilon  #transpose because items are columns

pd.DataFrame(itemFeatures)
pd.DataFrame(userFeatures)


,0,1,2,3,4,5,6,7,8,9
0,0.381895,0.120093,-0.183932,0.081855,-0.172495,-0.018537,0.085225,-0.033943,-0.026010,-0.040274
1,4.346500,-0.254511,1.188366,-0.603347,0.524038,0.401075,-0.049077,0.172810,0.298001,-0.962251
2,3.823635,-0.070926,-1.072989,-0.089419,-0.526561,0.197240,0.567256,0.125846,-0.099780,0.569202
3,5.570711,-1.224080,-0.016986,-0.819378,-0.032188,-0.798978,0.309825,0.504152,-0.830487,1.215074
4,3.633484,-0.287964,0.091954,0.323658,0.680051,-0.844876,0.371963,-0.129387,0.194260,0.090955
...,...,...,...,...,...,...,...,...,...,...
195,4.708236,-0.516013,-0.281564,0.556151,-0.127141,-0.292898,0.240255,-0.108103,-0.773101,-0.382228
196,2.633977,0.215616,0.050425,-0.468769,0.259584,0.177353,0.242239,-0.036518,-0.129318,0.478436
197,0.945274,0.006840,-0.073202,-0.083986,0.191913,-0.147563,0.152222,0.139596,0.189199,0.022638
198,3.646332,0.822625,-0.085251,0.663647,0.741712,1.113013,-0.191071,0.064750,-0.701883,0.195095


In [162]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=8, n_init=1000)

model = kmeans.fit_predict(userFeatures)
modelDF = pd.DataFrame(model).rename(columns={0: 'group'}, index=idxUserDict)
modelDF


,group
102,2
103,7
104,3
105,5
106,3
...,...
297,1
298,0
299,2
300,3


Budowanie profili użytkowników  

In [163]:
separator = ';'
newGroups = [
    'Gry komputerowe', 'Gry na konsole', 'Sprzęt RTV', 'Komputery',
    'Telefony i akcesoria'
]


def castCategoryPath(categoryPath):
    categories = categoryPath.split(separator)
    foundGroups = [group for group in newGroups if group in categories]
    if len(foundGroups) != 1:
        raise RuntimeError('wrong group cast: {}'.format(foundGroups))
    return foundGroups[0]

In [164]:
def build_user_profile(userID: int, merged: pd.DataFrame) -> dict:
    profile = {}
    userActDF = merged.loc[merged['user_id'] == userID].drop(
        columns=['user_id'])

    for group in newGroups:
        g = userActDF.loc[userActDF['category_path'] == group].drop(
            columns=['category_path']).value_counts().to_frame().rename(
                columns={
                    0: 'count'
                }).reset_index()
        g = g[g['count'] >= g['count'].quantile(0.9)]
        profile[group] = g.set_index('product_id').to_dict()
    return profile


In [207]:
merged = pd.merge(sessionsDF, productsDF, how='inner', on='product_id')
merged.drop(columns=[
    'session_id', "timestamp", "event_type", "offered_discount", "purchase_id",
    'product_name', 'price', 'user_rating'
],
            inplace=True)
merged['category_path'] = merged['category_path'].apply(castCategoryPath)

usersID = usersDF['user_id'].unique()
profileDict = {}

for userID in usersID:
    profileDict[userID] = build_user_profile(userID, merged)

#konieczne jest przepakowanie dict'a, gdyż konwersja z int64 na int ma w numpy'u buga, który dokonuje konwersji int64 na int32. Problem otwarty na Github'ie, tylko Windows.
profileDict = {int(key): value for key, value in profileDict.items()}


{102: {'Gry komputerowe': {'count': {1050: 14, 1053: 6, 1054: 6, 1049: 6}},
  'Gry na konsole': {'count': {1041: 2}},
  'Sprzęt RTV': {'count': {1233: 3}},
  'Komputery': {'count': {1017: 3, 1077: 3, 1276: 3}},
  'Telefony i akcesoria': {'count': {1317: 1}}},
 103: {'Gry komputerowe': {'count': {1050: 65,
    1053: 49,
    1049: 32,
    1048: 30,
    1052: 30,
    1055: 28,
    1056: 24,
    1057: 24,
    1054: 23,
    1051: 21,
    1083: 7,
    1272: 7,
    1241: 7}},
  'Gry na konsole': {'count': {1004: 16, 1084: 11, 1047: 10, 1031: 10}},
  'Sprzęt RTV': {'count': {1292: 8, 1306: 8, 1286: 7, 1288: 7}},
  'Komputery': {'count': {1017: 9, 1076: 7, 1079: 7}},
  'Telefony i akcesoria': {'count': {1319: 7}}},
 104: {'Gry komputerowe': {'count': {1053: 63,
    1050: 53,
    1048: 37,
    1055: 36,
    1056: 28,
    1049: 27,
    1054: 26,
    1052: 24,
    1272: 18,
    1241: 11,
    1051: 10}},
  'Gry na konsole': {'count': {1011: 14, 1006: 9, 1047: 9}},
  'Sprzęt RTV': {'count': {1286: 1

In [218]:
groupsLabels = modelDF['group'].unique()
groupsTop = {}

for label in groupsLabels:
    group = modelDF.loc[modelDF['group'] == label].index
    categoryDict = {}
    for category in newGroups:
        dfListPre = []
        for user in group:
            dfListPre.append(pd.DataFrame(profileDict[user][category]))

        dfList = [
            df.reset_index().rename(columns={'index': 'product_id'})
            for df in dfListPre
        ]

        df = pd.concat(dfList).groupby(['product_id']).sum().reset_index().sort_values(
            by=['count'], ascending=False)

        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['count'], inplace=True)
        categoryDict[category] = df['product_id'].to_list()

    groupsTop[label] = categoryDict

#konieczne jest przepakowanie dict'a, gdyż konwersja z int64 na int ma w numpy'u buga, który dokonuje konwersji int64 na int32. Problem otwarty na Github'ie, tylko Windows.
groupsTop = {int(key): value for key, value in groupsTop.items()}


Zapis modelu i profili do plików.

In [219]:
import json

modelDF.to_json('../models/similar_users.json')

with open('../models/profiles.json', 'w') as file:
    json.dump(profileDict, file, sort_keys=True, indent=4)

with open('../models/groups_profiles.json', 'w') as file:
    json.dump(groupsTop, file, sort_keys=True, indent=4)


Testowe odczytanie danych

In [167]:
savedModel = pd.read_json('../models/similar_users.json')

with open('../models/profiles.json', 'r') as file:
    data = json.load(file)

#json.load wczytuje klucze jako str, przepakowanie na int upraszcza pracę z modelem
savedProfiles = {int(key): value for key, value in data.items()}

display(savedModel)
savedProfiles[102]

,group
102,2
103,7
104,3
105,5
106,3
...,...
297,1
298,0
299,2
300,3


{'Gry komputerowe': {'count': {'1049': 6, '1050': 14, '1053': 6, '1054': 6}},
 'Gry na konsole': {'count': {'1041': 2}},
 'Komputery': {'count': {'1017': 3, '1077': 3, '1276': 3}},
 'Sprzęt RTV': {'count': {'1233': 3}},
 'Telefony i akcesoria': {'count': {'1317': 1}}}

Przykład predykcji  
Użytkownik o id 102 przegląda produkt z kategorii *Gry komputerowe*

Poniżej predykcja na modelu w pamięci.

In [168]:
input = [102, 'Gry komputerowe']

group = modelDF.loc[modelDF['group'] == modelDF.iloc[102]['group']].index
dfListPre = []

for user in group:
    dfListPre.append(pd.DataFrame(profileDict[user][input[1]]))

dfList = [
    df.reset_index().rename(columns={'index': 'product_id'})
    for df in dfListPre
]

df = pd.concat(dfList).groupby(['product_id']).sum().reset_index().sort_values(
    by=['count'], ascending=False)
df

,product_id,count
5,1053,1857
2,1050,1586
0,1048,1094
1,1049,722
7,1055,660
4,1052,634
6,1054,489
8,1056,473
3,1051,232
11,1272,155


Predykcja dla modelu odczytanego z .json.  
Predykcje są identyczne, stąd wnioskujemy poprawnośc serializacji.

In [169]:
input = [102, 'Gry komputerowe']

group = savedModel.loc[savedModel['group'] == savedModel.iloc[102]
                       ['group']].index
dfListPre = []

for user in group:
    dfListPre.append(pd.DataFrame(savedProfiles[user][input[1]]))

dfList = [
    df.reset_index().rename(columns={'index': 'product_id'})
    for df in dfListPre
]

df = pd.concat(dfList).groupby(['product_id']).sum().reset_index().sort_values(
    by=['count'], ascending=False)
df

,product_id,count
5,1053,1857
2,1050,1586
0,1048,1094
1,1049,722
7,1055,660
4,1052,634
6,1054,489
8,1056,473
3,1051,232
11,1272,155
